In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import gensim.downloader as api
import numpy as np
from time import time
import re
import nltk
nltk.download('stopwords')
import itertools
import string
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda,Dropout
from keras import backend as K
from keras.optimizers import Adadelta
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.layers import Dense
from keras.optimizers import SGD, Adam
import keras

word_embed_model = api.load('glove-wiki-gigaword-300')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[==============================================----] 92.6% 348.4/376.1MB downloaded

In [0]:
import pandas as pd

train_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/train4.csv')
dev_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/devset.csv')
test_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/testset3.csv')

print(train_df.shape)
print(dev_df.shape)
print(test_df.shape)

(150000, 3)
(120581, 6)
(584843, 6)


In [0]:
# Data preprocessing
def change_label(df):
  labels = df['label']
  new_labels = []
  for row in labels:
    if row == 'SUPPORTS':
      new_labels.append(2)
    elif row == 'REFUTES':
      new_labels.append(1)
    else:
      new_labels.append(0)
  df['label'] = pd.Series(new_labels)

def process_text(raw_text):
    text = str(raw_text)
    text = re.sub('_', ' ', text)
    text = re.sub('-LRB-', ' ', text)
    text = re.sub('-RRB-', ' ', text)
    text = re.sub('-LSB-', ' ', text)
    text = re.sub('-RSB-', ' ', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.split()
    return text

change_label(train_df)
change_label(dev_df)


In [0]:
vocab = {}
vocab_index = ['<unk>']

#substitute the sentence in the train data into the index representation
for index, row in train_df.iterrows():
  if index%100000 == 0:
    print(index)
  for sentence in ['claim', 'evidence']:
    number_rep = []
    for word in process_text(row[sentence]):
      if word in stopwords.words('english') and word not in word_embed_model.vocab:
        continue
      if word not in vocab:
        vocab[word] = len(vocab_index)
        number_rep.append(len(vocab_index))
        vocab_index.append(word)
      else:
        number_rep.append(vocab[word])
    train_df.at[index, sentence] = number_rep
    
#substitute the sentence in the test data into the index representation
for index, row in test_df.iterrows():
  if index%100000 == 0:
    print(index)
  for sentence in ['claim', 'evidence']:
    number_rep = []
    for word in process_text(row[sentence]):
      if word in stopwords.words('english') and word not in word_embed_model.vocab:
        continue
      if word not in vocab:
        vocab[word] = len(vocab_index)
        number_rep.append(len(vocab_index))
        vocab_index.append(word)
      else:
        number_rep.append(vocab[word])
    test_df.at[index, sentence] = number_rep
    
    
#embedding all vocabulary in a matrix
dimension = 300        
embeddings = 1 * np.random.randn(len(vocab) + 1,dimension)
embeddings[0] = 0 
for word, index in vocab.items():
  if word in word_embed_model.vocab:
    embeddings[index] = word_embed_model[word]

change_label(dev_df)

0
100000
0
100000
200000
300000
400000
500000


In [0]:
#substitute the sentence in the train data into the index representation
for index, row in dev_df.iterrows():
  if index%100000 == 0:
    print(index)
  for sentence in ['claim', 'evidence']:
    number_rep = []
    for word in process_text(row[sentence]):
      if word in stopwords.words('english') and word not in word_embed_model.vocab:
        continue
      if word not in vocab:
        vocab[word] = len(vocab_index)
        number_rep.append(len(vocab_index))
        vocab_index.append(word)
      else:
        number_rep.append(vocab[word])
    dev_df.at[index, sentence] = number_rep
    

0
100000


In [0]:
test2_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/new_test.csv')

#substitute the sentence in the train data into the index representation
for index, row in test2_df.iterrows():
  if index%100000 == 0:
    print(index)
  for sentence in ['claim', 'evidence']:
    number_rep = []
    for word in process_text(row[sentence]):
      if word in stopwords.words('english') and word not in word_embed_model.vocab:
        continue
      if word not in vocab:
        vocab[word] = len(vocab_index)
        number_rep.append(len(vocab_index))
        vocab_index.append(word)
      else:
        number_rep.append(vocab[word])
    test2_df.at[index, sentence] = number_rep
test2_df.to_csv("new_test_emb.csv")


0
100000


In [0]:
def max_sequence(df1,df2):
    max_seq = max(df1.claim.map(lambda x: len(x)).max(), df1.evidence.map(lambda x: len(x)).max(),df2.claim.map(lambda x: len(x)).max(), df2.evidence.map(lambda x: len(x)).max())
    return max_seq

In [0]:
# Build a neural network
from keras.optimizers import RMSprop
from keras.layers import Concatenate
def manhattan_distance(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))
n_hidden = 100
n_epoch = 100
batch_size = 2048

max_seq = max_sequence(train_df, test_df)
left_input = Input(shape=(max_seq,), dtype='int32')
right_input = Input(shape=(max_seq,), dtype='int32')
embedding_layer = Embedding(len(embeddings), dimension, weights=[embeddings], input_length=max_seq, trainable=False)
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)
shared_lstm = LSTM(n_hidden)
left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

malstm_distance = Lambda(function=lambda x: manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])
con = Concatenate()([left_output, right_output, malstm_distance])
dense1 = Dense(128, activation='relu', input_dim=201)(con)
drop1 = Dropout(0.5)(dense1)
dense2 = Dense(3, activation='softmax')(drop1)
model =Model([left_input, right_input], dense2)
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

In [0]:
# Preparing training data
cols = ['claim','evidence']
x = train_df[cols]
y = train_df['label']
x_val = dev_df[cols]
y_val = dev_df['label']
# v_size = 15000
# t_size = len(train_df) - v_size
    
# x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=v_size)
x = {'claim': x.claim, 'evidence': x.evidence}
x_val = {'claim': x_val.claim, 'evidence': x_val.evidence}

y = y.values
y_val = y_val.values

for dataset, side in itertools.product([x, x_val], ['claim', 'evidence']):
  dataset[side] = pad_sequences(dataset[side], maxlen=max_seq)

In [0]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(y), y)

y = keras.utils.to_categorical(y, num_classes=3, dtype ='int32')

y_val = keras.utils.to_categorical(y_val, num_classes=3)

In [0]:
# Draw pychart
from keras.callbacks import Callback 
import matplotlib.pyplot as plt 
import matplotlib 

matplotlib.use('Agg') 
class LossHistory(Callback): 

   def on_train_begin(self, logs={}): 
       self.losses = {'batch':[], 'epoch':[]} 
       self.accuracy = {'batch':[], 'epoch':[]} 
       self.val_loss = {'batch':[], 'epoch':[]} 
       self.val_acc = {'batch':[], 'epoch':[]} 

   def on_batch_end(self, batch, logs={}): 
       self.losses['batch'].append(logs.get('loss')) 
       self.accuracy['batch'].append(logs.get('acc')) 
       self.val_loss['batch'].append(logs.get('val_loss')) 
       self.val_acc['batch'].append(logs.get('val_acc')) 

   def on_epoch_end(self, batch, logs={}): 
       self.losses['epoch'].append(logs.get('loss')) 
       self.accuracy['epoch'].append(logs.get('acc')) 
       self.val_loss['epoch'].append(logs.get('val_loss')) 
       self.val_acc['epoch'].append(logs.get('val_acc')) 

   def loss_plot(self, savepath): 
       iters = range(len(self.losses["epoch"])) 
       plt.figure() 
       # acc 
       plt.plot(iters, self.accuracy["epoch"], 'r', label='train acc') 
       # loss 
       plt.plot(iters, self.losses["epoch"], 'g', label='train loss') 
       
       plt.grid(True) 
       plt.xlabel('epoch') 
       plt.ylabel('acc-loss') 
       plt.legend(loc="upper right")  
       plt.savefig(savepath) 
    
   def val_loss_plot(self, savepath): 
   
       iters = range(len(self.losses["epoch"])) 
       plt.figure() 
      
       # val_acc 
       plt.plot(iters, self.val_acc["epoch"], 'b', label='val acc') 
       # val_loss 
       plt.plot(iters, self.val_loss["epoch"], 'k', label='val loss') 
       plt.grid(True) 
       plt.xlabel('epoach') 
       plt.ylabel('acc-loss') 
       plt.legend(loc="upper right")  
       plt.savefig(savepath) 

In [0]:
# Training
start = time()
history = LossHistory()
model_checkpoint = ModelCheckpoint('/content/gdrive/My Drive/yinghao/yinghao.hdf5', monitor='loss', verbose=1, save_best_only=True)
train = model.fit([x["claim"], x["evidence"]], y, epochs=n_epoch, batch_size=batch_size)
history.loss_plot("/content/gdrive/My Drive/yinghao/train.png")
history.val_loss_plot("/content/gdrive/My Drive/yinghao/val.png")
print(time()-start)

NameError: ignored

In [0]:
# Prediction
x_test = test2_df[cols]
x_test = {'claim': x_test.claim, 'evidence': x_test.evidence}
for dataset, side in itertools.product([x_test], ['claim', 'evidence']):
  dataset[side] = pad_sequences(dataset[side], maxlen=max_seq)
  
predict = model.predict([x_test["claim"], x_test["evidence"]],batch_size=2048)
print(predict.shape)


(107244, 3)


In [0]:
def find_max_pos(array):
  m = max(array)
  p = 0
  for i in range(0,len(array)):
    if (array[i] == m):
      p = i
  return p


In [0]:
test_label = []
for array in predict:
  pos = find_max_pos(array)
  test_label.append(pos)

print(len(test_label))

107244


In [0]:
from collections import Counter

print(Counter(test_label))

Counter({0: 60787, 1: 23823, 2: 22634})


In [0]:
# generate json file for test set
r = {}
cur_key = ""
for index, row in result_df.iterrows():
  # just update evidence when the key does not change
  if row['key'] == cur_key:
    tmp = []
    tmp.append(row['title'])
    tmp.append(int(row['page']))
    tmp.append(row['label'])
    r[cur_key]['evidence'].append(tmp)
  
  # when key changed, we need add new member
  else:
    cur_key = row['key']
    r[cur_key] = {}
    if row['evidence'] == 'None':
      r[cur_key]['claim'] = r[cur_key].setdefault('claim', row['claim'])
      r[cur_key]['evidence'] =  r[cur_key].setdefault('evidence', [])
        
    else:
      r[cur_key]['claim'] = r[cur_key].setdefault('claim', row['claim'])
      r[cur_key]['evidence'] =  r[cur_key].setdefault('evidence', [])
      tmp = []
      tmp.append(row['title'])
      tmp.append(int(row['page']))
      tmp.append(row['label'])
      r[cur_key]['evidence'].append(tmp)

In [0]:
keys = r.keys()
for key in keys:
  # remove not enough info
  if len(r[key]['evidence']) > 0:
    for item in r[key]['evidence'].copy():
      # if tagged not enough then remove
      if item[2] == 0:
        r[key]['evidence'].remove(item)
    
    if len(r[key]['evidence']) == 0:
      r[key]['label'] = r[key].setdefault('label', "NOT ENOUGH INFO")
      
    else:
      temp = set()
      for item in r[key]['evidence']:
        temp.add(item[2])
      if len(temp) == 2:
        r[key]['label'] = r[key].setdefault('label', "NOT ENOUGH INFO")
        r[key]['evidence'] = []
      elif len(temp) == 1:
        l = list(temp)[0]
        if l == 1:
          r[key]['label'] = "REFUTES"
          for i in range(len(r[key]['evidence'])):
            r[key]['evidence'][i] = r[key]['evidence'][i][:2]
        elif l == 2:
          r[key]['label'] =  "SUPPORTS"
          for i in range(len(r[key]['evidence'])):
            r[key]['evidence'][i] = r[key]['evidence'][i][:2]
  else:
    r[key]['label'] = r[key].setdefault('label', "NOT ENOUGH INFO")

In [0]:
import json
json_str = json.dumps(r, indent=4)
with open('newoutput.json', 'w') as json_file:
    json_file.write(json_str)

In [0]:
tmp = json.load(open('newoutput.json','r'))
print(len(list(tmp.keys())))

14997
